In [2]:
import numpy as np
import pandas as pd

# Data understanding

In [3]:
df_sales = pd.read_csv("../data/raw/sales_train.csv")
df_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
df_items = pd.read_csv("../data/raw/items.csv")
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
df_item_categories = pd.read_csv("../data/raw/item_categories.csv")
df_item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [6]:
df_shops = pd.read_csv("../data/raw/shops.csv")
df_shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [7]:
df_test = pd.read_csv("../data/raw/test.csv")
print(f"Shape: {df_test.shape}")
df_test.head()

Shape: (214200, 3)


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


**First Impressions**

- as stated in challenge's description, the test set has only one month while the train set has days/day/item.
- as stated in challenge's description, and at first glance, the other provided files do not add any value to the forecast analysis.

## Exploring df_sales

In [8]:
df_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [9]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [10]:
df_sales["year_month"] = df_sales["date"].str.slice(start=-4) + df_sales["date"].str.slice(start=3, stop=5)
df_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year_month
0,02.01.2013,0,59,22154,999.00,1.0,201301
1,03.01.2013,0,25,2552,899.00,1.0,201301
2,05.01.2013,0,25,2552,899.00,-1.0,201301
3,06.01.2013,0,25,2554,1709.05,1.0,201301
4,15.01.2013,0,25,2555,1099.00,1.0,201301


In [11]:
print(f"""Unique values:
      - year_month: {df_sales["year_month"].nunique()}
      - date_block_num: {df_sales["date_block_num"].nunique()}
      - shop_id: {df_sales["shop_id"].nunique()}
      - item_id: {df_sales["item_id"].nunique()}""")

Unique values:
      - year_month: 34
      - date_block_num: 34
      - shop_id: 60
      - item_id: 21807


## Exploring df_test

In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB


In [13]:
print(f"""Unique values:
      - shop_id: {df_test["shop_id"].nunique()}
      - item_id: {df_test["item_id"].nunique()}""")

Unique values:
      - shop_id: 42
      - item_id: 5100


## Verifying if the values in df_test exist in df_sales

In [23]:
df = df_test.groupby(by=["shop_id", "item_id"]).agg({"ID":["count", "nunique"]})
df.head(60)

ID        
                count nunique
shop_id item_id              
2       30          1       1
        31          1       1
        32          1       1
        33          1       1
        38          1       1
        42          1       1
        45          1       1
        51          1       1
        53          1       1
        57          1       1
        59          1       1
        62          1       1
        64          1       1
        67          1       1
        69          1       1
        70          1       1
        72          1       1
        76          1       1
        80          1       1
        83          1       1
        88          1       1
        91          1       1
        93          1       1
        109         1       1
        138         1       1
        140         1       1
        149         1       1
        153         1       1
        154         1       1
        166         1       1
        168         1       1
        173         1       1
        180         1       1
        202         1       1
        204         1       1
        205         1       1
        213         1       1
        226         1       1
        234         1       1
        237         1       1
        248         1       1
        249         1       1
        253         1       1
        254         1       1
        259         1       1
        269         1       1
        302         1       1
        308         1       1
        313         1       1
        315         1       1
        325         1       1
        342         1       1
        353         1       1
        356         1       1
        388         1       1
        402         1       1
        407         1       1
        408         1       1
        413         1       1
        421         1       1

In [26]:
df[("ID")].value_counts()

count  nunique
1      1          214200
dtype: int64